In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import calendar
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pandas")
import os

def save_raw_dividend_data(ticker_list, filename="raw_dividends.csv"):
    """
    yfinance에서 티커별 배당 데이터를 수집하여 하나의 CSV로 저장합니다.
    - 타임존 제거 (tz-naive)
    - 병합된 DataFrame을 'Date' 인덱스로 저장
    - 컬럼: 각 티커별 배당금 시계열
    """
    dividend_data = {}

    for ticker in ticker_list:
        try:
            stock = yf.Ticker(ticker)
            div = stock.dividends
            div.index = div.index.tz_localize(None)

            if div.empty:
                print(f"⚠️ 배당 데이터 없음: {ticker}")
                continue

            dividend_data[ticker] = div

        except Exception as e:
            print(f"{ticker} 처리 중 오류: {e}")

    # 병합: 인덱스를 날짜로 통일
    if not dividend_data:
        print("수집된 데이터가 없습니다.")
        return None

    df = pd.DataFrame(dividend_data)
    df.index.name = "Date"
    df.sort_index(inplace=True)

    # 저장
    df.to_csv(filename, encoding='utf-8-sig')
    print(f"원천 배당 데이터 저장 완료: {filename}")
    return df


def calculate_annualized_yield_timeseries(raw_dividends: pd.DataFrame, raw_prices: pd.DataFrame):
    """
    원천 배당 데이터와 주가 데이터를 기반으로 티커별 연환산 배당률 시계열을 생성합니다.
    - 배당 지급일 기준으로 12개월간 배당 횟수 계산
    - 해당 월의 마지막 날 주가 기준으로 연환산 배당률 계산
    - 결과는 딕셔너리 형태로 반환: {티커: DataFrame}
    """
    result = {}

    for ticker in raw_dividends.columns:
        if ticker not in raw_prices.columns:
            print(f"⚠️ {ticker}의 가격 데이터가 없습니다.")
            continue

        div_series = raw_dividends[ticker].dropna()
        price_series = raw_prices[ticker].dropna()

        ts_result = []

        for date, dividend in div_series.items():
            # 🧠 주가 날짜 보정 (주말/휴일 문제 방지)
            if date not in price_series.index:
                # 가장 가까운 거래일로 보정
                shifted_date = price_series.index[price_series.index.get_indexer([date], method='nearest')[0]]
            else:
                shifted_date = date

            year = shifted_date.year
            month = shifted_date.month
            last_day = calendar.monthrange(year, month)[1]
            shifted_date = shifted_date.replace(day=last_day)

            # ✅ 해당 달의 마지막 날로 변환
            start_date = shifted_date - relativedelta(months=12)
            div_last_year = div_series[(div_series.index > start_date) & (div_series.index <= shifted_date)]
            dividend_count = len(div_last_year)

            if dividend_count == 0:
                continue

            price = price_series.get(shifted_date)
            if price is None or price == 0:
                continue

            annual_yield = (dividend * dividend_count / price) * 100

            ts_result.append({
                "Date": date,
                "Dividend": dividend,
                "Price": price,
                "Dividend Count": dividend_count,
                "Annualized Yield (%)": round(annual_yield, 2)
            })

        if ts_result:
            df = pd.DataFrame(ts_result).set_index("Date")
            result[ticker] = df
        else:
            print(f"ℹ️ {ticker}는 유효한 배당 시계열이 없습니다.")

    return result

def load_dividend_data(ticker_list):
    """
    저장된 티커별 배당률 시계열 파일들을 불러와 딕셔너리 형태로 반환합니다.
    - 경로: dividend_tickers/{티커}.csv
    - 반환 형식: {티커: DataFrame}
    """
    dividend_data = {}
    for ticker in ticker_list:
        try:
            df = pd.read_csv(f"dividend_tickers/{ticker}.csv", index_col=0, parse_dates=True, encoding='utf-8-sig')
            dividend_data[ticker] = df
        except FileNotFoundError:
            print(f"⚠️ {ticker}의 배당 데이터 파일을 찾을 수 없습니다.")
    return dividend_data


def calculate_portfolio_yield_timeseries_from_dict(dividend_data: dict, weights: dict):
    """
    미리 불러온 티커별 배당률 시계열 데이터와 포트폴리오 비중을 바탕으로
    포트폴리오의 가중 평균 배당률 시계열을 계산합니다.
    - 비중이 부여된 각 티커의 'Annualized Yield (%)'를 합산
    - ffill로 결측값 보완
    - 결과: DataFrame (각 티커 + Portfolio Yield 포함)
    """
    dfs = []

    for ticker, weight in weights.items():
        if ticker not in dividend_data:
            print(f"⚠️ {ticker}는 불러온 배당 데이터에 없습니다.")
            continue

        df = dividend_data[ticker]
        if "Annualized Yield (%)" not in df.columns:
            print(f"⚠️ {ticker} 데이터에 'Annualized Yield (%)' 컬럼이 없습니다.")
            continue

        df = df[["Annualized Yield (%)"]].dropna()
        df["Weighted Yield"] = df["Annualized Yield (%)"] * weight
        dfs.append(df[["Weighted Yield"]].rename(columns={"Weighted Yield": ticker}))

    if not dfs:
        print("⛔ 유효한 티커 데이터가 없습니다.")
        return None

    # 날짜 기준 병합 후 가중합
    combined_df = pd.concat(dfs, axis=1)
    combined_df = combined_df.ffill()
    combined_df.index.name = "Date"
    combined_df["Portfolio Yield (%)"] = combined_df.sum(axis=1)

    return combined_df


In [ ]:
## 원천데이터 크롤링

In [55]:
ticker_list = ['SCHD', 'JEPQ', 'JEPI', 'TLT', 'SHY', 'SGOV', 'QQQ', 'SPY', 
               'VYM', 'HDV', 'DVY', 'NOBL', 'DHS', 'SPYD', 'QYLD', 'RYLD', 'XYLD', 'XYLD', 'PFFD']

In [30]:
raw_dividends = save_raw_dividend_data(ticker_list, filename="raw_dividends.csv")

원천 배당 데이터 저장 완료: raw_dividends.csv


In [ ]:
raw_price = yf.download(ticker_list, ignore_tz=True, auto_adjust=False)
raw_price['Close'].to_csv("raw_price.csv", encoding='utf-8-sig')

[*********************100%***********************]  18 of 18 completed


In [ ]:
## 원천데이터 불러오기

In [31]:
raw_dividends = pd.read_csv("raw_dividends.csv", index_col=0, parse_dates=True, encoding='utf-8-sig')
raw_prices = pd.read_csv("raw_price.csv", index_col=0, parse_dates=True, encoding='utf-8-sig')

In [ ]:
## 연환산 배당률 시계열 계산

In [77]:
result = calculate_annualized_yield_timeseries(raw_dividends, raw_prices)

In [83]:
for key in result.keys():
    result[key].to_csv(f"dividend_tickers/{key}.csv", encoding='utf-8-sig')

In [ ]:
## 포트폴리오 배당률 시계열 계산

In [132]:
tickers = ['SCHD', 'JEPI', 'QYLD']
dfs = load_dividend_data(tickers)
weights = {
    "SCHD": 0.3,
    "JEPI": 0.4,
    "QYLD": 0.3
}

combined_df = calculate_portfolio_yield_timeseries_from_dict(dfs, weights)

In [ ]:
## 목표자금 계산

In [137]:
target_dividend = combined_df['Portfolio Yield (%)'].iloc[-1]

In [136]:
target_cashflow = 200 * 12

In [138]:
target_cashflow/(target_dividend*0.01*(1-0.15))

40503.936476326286

In [139]:
target_dividend

6.971